In [1]:
import pandas as pd
import numpy as np


df = pd.read_csv('data/COMBO_V3.tsv', sep='\t')
mp = dict(zip(df['Protein.Group'], df['Genes']))
df = df[df['Lib.PG.Q.Value'] <= 0.01]
## normalization
df.drop_duplicates(['Stripped.Sequence', 'File.Name'], inplace=True)
mp = dict(zip(df['Protein.Group'], df['Genes']))
# now sum across charge states
pep_level = df.groupby(['Stripped.Sequence', 'Protein.Group', 'File.Name']).apply(
    lambda x: np.nansum(x['Precursor.Quantity'])).reset_index()
pep_level = pd.pivot_table(pep_level, values=0, index=[
    'Protein.Group', 'Stripped.Sequence'], columns='File.Name')

File.Name                              Z:\data\timstof-backup\RAW\2021\C001_RA1_1_8057.d  \
Protein.Group     Stripped.Sequence                                                        
A0A075B6I0        TLIYSTNTR                                                          NaN   
A0A075B6J1-20-123 QPVLTQPPSSSASPGESAR                                                NaN   
                  YLLYYYSDSDK                                                        NaN   
A0A075B6J9        VSWYQQPPGTAPK                                                  3698.03   
A0A075B6K2        FSGSNPGNTATLTISR                                                   NaN   
...                                                                                  ...   
Q9Y6R7-24-5405    SVPGCEGVALVVAQTK                                                   NaN   
Q9Y6Z7            DEAANTLIADYVAK                                                 1542.01   
                  FVGQLDISIAR                                                    7131.05   
                  FYYIVQEEK                                                      4009.03   
                  VFIGVNDLER                                                         NaN   

File.Name                              Z:\data\timstof-backup\RAW\2021\C002_RA2_1_8058.d  \
Protein.Group     Stripped.Sequence                                                        
A0A075B6I0        TLIYSTNTR                                                          NaN   
A0A075B6J1-20-123 QPVLTQPPSSSASPGESAR                                                NaN   
                  YLLYYYSDSDK                                                        NaN   
A0A075B6J9        VSWYQQPPGTAPK                                                      NaN   
A0A075B6K2        FSGSNPGNTATLTISR                                                   NaN   
...                                                                                  ...   
Q9Y6R7-24-5405    SVPGCEGVALVVAQTK                                                   NaN   
Q9Y6Z7            DEAANTLIADYVAK                                                 2878.03   
                  FVGQLDISIAR                                                    2357.02   
                  FYYIVQEEK                                                      3084.02   
                  VFIGVNDLER                                                     4824.03   

File.Name                              Z:\data\timstof-backup\RAW\2021\C003_BA3_1_8049.d  \
Protein.Group     Stripped.Sequence                                                        
A0A075B6I0        TLIYSTNTR                                                          NaN   
A0A075B6J1-20-123 QPVLTQPPSSSASPGESAR                                                NaN   
                  YLLYYYSDSDK                                                        NaN   
A0A075B6J9        VSWYQQPPGTAPK                                                      NaN   
A0A075B6K2        FSGSNPGNTATLTISR                                                   NaN   
...                                                                                  ...   
Q9Y6R7-24-5405    SVPGCEGVALVVAQTK                                                   NaN   
Q9Y6Z7            DEAANTLIADYVAK                                                     NaN   
                  FVGQLDISIAR                                                        NaN   
                  FYYIVQEEK                                                          NaN   
                  VFIGVNDLER                                                         NaN   

File.Name                              Z:\data\timstof-backup\RAW\2021\C003_RA3_1_8059.d  \
Protein.Group     Stripped.Sequence                                                        
A0A075B6I0        TLIYSTNTR                                                          NaN   
A0A075B6J1-20-123 QPVLTQPPSSSASPGESAR                                                NaN   
                  YLLYYYSDSDK            

In [2]:

#from mypy import biostat as bst

def remove_dups(df):

    ## a bit hacky but needed a way to remove double runs ###
    pids = df.count().to_frame()
    pids['ids'] = [
        x.split('\\')[-1].replace('.d', '').split('_')[0] for x in list(pids.index)]
    pids.reset_index(inplace=True)
    ## now we keep unique max value per group
    pids = pids.loc[pids.reset_index().groupby(
        ['ids'])[0].idxmax()].reset_index()
    df = df.iloc[:, pids['level_0']]
    return df


def remove_outliers(df):
    '''
    remove outlier in wide format data by count the nr of missing values
    (peplevel)
    '''
    from scipy import stats
    df = df.count().to_frame()
    df['zscore'] = stats.zscore(df.values, axis=None)
    df = df[df['zscore'] > -1.5]
    return list(df.index)


def fetch_meta():
    # metadata
    cols = "#E64B35B2", "#4DBBD5B2", "#00A087B2", "#3C5488B2", "#F39B7FB2", "#8491B4B2", "#91D1C2B2", "#DC0000B2", "#7E6148B2"
    cols = [x.lower() for x in list(cols)]

    meta = pd.read_excel('meta/COMBO PRO Banked Plasma Samples Clinical.xlsx')
    dd = list(set(meta['COMBO Plasma Box Number']))
    enc = dict(zip(dd, cols))
    col_colors = meta['COMBO Plasma Box Number'].map(enc)

    col_colors.index = meta['COMBO ID']
    meta['COMBO Plasma Box Number'] = pd.Categorical(
        meta['COMBO Plasma Box Number'])

    meta['code'] = meta['COMBO Plasma Box Number'].cat.codes
    return meta, col_colors


def preprocess_metadata(flt):
    # metadata
    cols = "#E64B35B2", "#4DBBD5B2", "#00A087B2", "#3C5488B2", "#F39B7FB2", "#8491B4B2", "#91D1C2B2", "#DC0000B2", "#7E6148B2"
    cols = [x.lower() for x in list(cols)]

    meta = pd.read_excel('meta/COMBO PRO Banked Plasma Samples Clinical.xlsx')
    dd = list(set(meta['COMBO Plasma Box Number']))
    meta = meta[meta['COMBO ID'].isin(flt)]
    enc = dict(zip(dd, cols))
    col_colors = meta['COMBO Plasma Box Number'].map(enc)

    col_colors.index = meta['COMBO ID']
    meta['COMBO Plasma Box Number'] = pd.Categorical(
        meta['COMBO Plasma Box Number'])

    meta['code'] = meta['COMBO Plasma Box Number'].cat.codes
    return meta, col_colors




pep_level.columns = [x.split('\\')[-1].replace('.d', '').split('_')[0]
                     for x in list(pep_level)]
pep_level = pep_level.drop('C028', axis=1)
pep_level = remove_dups(pep_level)
idx = remove_outliers(pep_level)
pep_level = pep_level[idx]
meta, _ = preprocess_metadata(list(pep_level))

## normalize per batch

# # 
# toadd = []
# for x in set(meta['COMBO Plasma Box Number']):
#     subm = meta[meta['COMBO Plasma Box Number']==x]
#     norm_data = bst.median_polish(pep_level[list(subm['COMBO ID'])])['df'].reset_index()
#     norm_data = pd.melt(norm_data, id_vars=['Protein.Group', 'Stripped.Sequence'])
#     try:
#         norm_data.empty
#         toadd.append(norm_data)
#     except TypeError as e:
#         pass
# norm_data = pd.concat(toadd)
# norm_data = pd.pivot_table(norm_data, columns='variable', index=['Protein.Group', 'Stripped.Sequence'], values='value')

## try to normalize with median centering of all peptides identified everywhere

normf = pep_level.dropna(thresh=pep_level.shape[1])
normf_md = normf.mean()
norm_data = (pep_level/normf_md.values) * np.median(normf_md.values)
norm_data = np.log2(norm_data)
# norm_data.to_csv('test.csv')

In [3]:


### NAN imputation

# remove stuff in less than 10% of the samples
norm_data_f = norm_data.dropna(thresh=round(norm_data.shape[1]*0.1))

nrn = norm_data_f.count(axis=1)
nrn = nrn/norm_data_f.shape[1]
nrn.to_csv('test.csv')
## impute with bst_imputnan
sub1 = nrn[nrn < 0.3]
norm_data_f1 = norm_data_f[norm_data_f.index.isin(sub1.index)]
norm_data_f1 = bst.impute_nan(norm_data_f1, q=0.3)

# impute with median
sub1 = nrn[nrn > 0.3]
norm_data_f2 = norm_data_f[norm_data_f.index.isin(sub1.index)].T
for x in list(norm_data_f2):
    norm_data_f2[x].fillna(norm_data_f2[x].median(skipna=True), inplace=True)

norm_data_f2 = norm_data_f2.T
norm_data = pd.concat([norm_data_f1, norm_data_f2])
norm_data

C001       C002       C003  \
Protein.Group     Stripped.Sequence                                    
A0A075B6J1-20-123 YLLYYYSDSDK         9.933630   8.393284   8.391173   
A0A075B6Q5        AEDMAVYYCAR        10.280366  10.913820   9.241812   
A0A075B6S5        ASQGISNYLAWYQQK     8.692348   8.947474   9.864499   
A0A087WSY6        ASQSVSSNLAWYQQK     9.810215  13.027864  10.014912   
A0A087WSY6-21-115 LLIYGASIR           8.675010  10.715927   8.558668   
...                                        ...        ...        ...   
Q9Y6R7-24-5405    LPVVLANGQIR         9.487376  10.450957  10.023390   
Q9Y6Z7            DEAANTLIADYVAK     10.391460  11.484431  10.770808   
                  FVGQLDISIAR        12.600762  11.196314  11.762426   
                  FYYIVQEEK          11.769901  11.584162  11.997424   
                  VFIGVNDLER         12.102486  12.229589  11.582922   

                                          C004       C005       C006  \
Protein.Group     Stripped.Sequence                                    
A0A075B6J1-20-123 YLLYYYSDSDK         9.382467   8.562358   8.492471   
A0A075B6Q5        AEDMAVYYCAR         8.641737   9.309725   8.224726   
A0A075B6S5        ASQGISNYLAWYQQK     9.336514   7.973316   9.697990   
A0A087WSY6        ASQSVSSNLAWYQQK    13.618755   8.790140   9.065069   
A0A087WSY6-21-115 LLIYGASIR           9.451582   7.367099   9.239869   
...                                        ...        ...        ...   
Q9Y6R7-24-5405    LPVVLANGQIR        11.263954  11.395751  11.990406   
Q9Y6Z7            DEAANTLIADYVAK     11.490233  11.436058  10.201105   
                  FVGQLDISIAR        13.118348  12.745018  12.744356   
                  FYYIVQEEK          11.997424  11.755094  11.214059   
                  VFIGVNDLER         11.543770  12.550463  12.190315   

                                          C007       C008       C009  \
Protein.Group     Stripped.Sequence                                    
A0A075B6J1-20-123 YLLYYYSDSDK         8.249872  10.488138   7.680657   
A0A075B6Q5        AEDMAVYYCAR        10.835988   9.922300   9.737162   
A0A075B6S5        ASQGISNYLAWYQQK    11.984036  12.985679   8.263064   
A0A087WSY6        ASQSVSSNLAWYQQK     9.439880  12.648802   7.981789   
A0A087WSY6-21-115 LLIYGASIR           8.722658   8.703618   8.995761   
...                                        ...        ...        ...   
Q9Y6R7-24-5405    LPVVLANGQIR        11.309607  10.984901  10.996093   
Q9Y6Z7            DEAANTLIADYVAK     10.944921  11.703423  11.532716   
                  FVGQLDISIAR        12.360777  10.983360  12.651462   
                  FYYIVQEEK          11.997424  11.997424  11.997424   
                  VFIGVNDLER         12.825175  12.102486  12.102486   

                                          C010  ...       C662       C663  \
Protein.Group     Stripped.Sequence             ...                         
A0A075B6J1-20-123 YLLYYYSDSDK         9.212162  ...   9.864880   9.106143   
A0A075B6Q5        AEDMAVYYCAR         8.761068  ...  12.175977  10.219912   
A0A075B6S5        ASQGISNYLAWYQQK     8.950473  ...  12.671970   9.082806   
A0A087WSY6        ASQSVSSNLAWYQQK     9.864362  ...   9.722138   9.668912   
A0A087WSY6-21-115 LLIYGASIR           9.530470  ...   9.810368   9.042337   
...                                        ...  ...        ...        ...   
Q9Y6R7-24-5405    LPVVLANGQIR        11.263954  ...  11.263954  11.263954   
Q9Y6Z7            DEAANTLIADYVAK     11.826119  ...  12.363257  11.818965   
                  FVGQLDISIAR        12.190388  ...  12.029037  12.045220   
                  FYYIVQEEK          11.997424  ...  12.052757  12.058463   
                  VFIGVNDLER         12.102486  ...  11.555748  12.056020   

                                          C664       C665       C666  \
Protein.Group     Stripped.Sequence                                    
A0A075B6J1-20-123 YLLYYYSDSDK        10.690352   7.972411   8.588863   
A0A075B6Q5   

In [4]:
from mypy import combat as cmbt


def preprocess_metadata(flt):
    # metadata
    cols = "#E64B35B2", "#4DBBD5B2", "#00A087B2", "#3C5488B2", "#F39B7FB2", "#8491B4B2", "#91D1C2B2", "#DC0000B2", "#7E6148B2"
    cols = [x.lower() for x in list(cols)]

    meta = pd.read_excel('meta/COMBO PRO Banked Plasma Samples Clinical.xlsx')
    dd = list(set(meta['COMBO Plasma Box Number']))
    meta = meta[meta['COMBO ID'].isin(flt)]
    enc = dict(zip(dd, cols))
    col_colors = meta['COMBO Plasma Box Number'].map(enc)

    col_colors.index = meta['COMBO ID']
    for x in ['COMBO Plasma Box Number', 'TB Classification', 'HIV Status']:
        meta[x] = pd.Categorical(meta[x])

    meta['code'] = meta['COMBO Plasma Box Number'].cat.codes
    meta['tbcode'] = meta['TB Classification'].cat.codes
    meta['hivcode'] = meta['HIV Status'].cat.codes
    
    return meta, col_colors


meta, _ = preprocess_metadata(list(norm_data))
mapping = dict(zip(meta['COMBO ID'], meta['code']))
batch = [int(mapping[x]+1) for x in list(norm_data) if x in mapping]
mod1 = [int(mapping[x]+1) for x in dict(zip(meta['COMBO ID'], meta['tbcode'])) if x in mapping]
mod2 = [int(mapping[x]+1) for x in dict(zip(meta['COMBO ID'], meta['hivcode'])) if x in mapping]
df_corr = cmbt.pycombat(norm_data, batch=np.array(batch))
df_corr.reset_index(inplace=True)
df_corr=norm_data.reset_index()
df_corr.drop('Stripped.Sequence', axis=1, inplace=True)

Found 8 batches.
Adjusting for 0 covariate(s) or covariate level(s).
Standardizing Data across genes.
Fitting L/S model and finding priors.
Finding parametric adjustments.


/Users/andreafossati/.local/lib/python3.8/site-packages/mypy-0.0.1-py3.8.egg/mypy/combat.py:334: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
/Users/andreafossati/.local/lib/python3.8/site-packages/mypy-0.0.1-py3.8.egg/mypy/combat.py:141: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
/Users/andreafossati/.local/lib/python3.8/site-packages/mypy-0.0.1-py3.8.egg/mypy/combat.py:137: RuntimeWarning: divide by zero encountered in true_divide


Adjusting the Data


In [5]:
def topN(x, n=3):
    if x.shape[0]==1:
        return x.mean().values
    elif x.shape[0] < n+1 and x.shape[0]>1:
        return x.mean().values
    else:
        tosum = x.mean(axis=1).sort_values(ascending=False).head(n)
        return x[x.index.isin(tosum.index)].mean().values

prot_level = df_corr.groupby('Protein.Group').apply(topN).to_frame()
prot_level = pd.DataFrame(prot_level[0].to_list(), columns=list(df_corr)[1:], index=prot_level.index)
prot_level.reset_index(inplace=True)
prot_level_mlt = pd.melt(prot_level, id_vars='Protein.Group')
prot_level_mlt = pd.merge(prot_level_mlt, meta,
                        left_on='variable', right_on='COMBO ID')
prot_level_mlt.reset_index(inplace=True)
prot_level_mlt['Genes'] = prot_level_mlt['Protein.Group'].map(mp)
prot_level_mlt.to_csv('processed/protein_level_melted_merged.csv', index=False)
prot_level_mlt

/var/folders/6w/j26fwf3s1gxbgknbxwsd30xh0000gn/T/ipykernel_3588/2115657204.py:3: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  return x.mean().values
/var/folders/6w/j26fwf3s1gxbgknbxwsd30xh0000gn/T/ipykernel_3588/2115657204.py:5: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  return x.mean().values
/var/folders/6w/j26fwf3s1gxbgknbxwsd30xh0000gn/T/ipykernel_3588/2115657204.py:7: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  tosum = x.mean(axis=1).sort_values(ascending=False).head(n)
/va

,index,Protein.Group,variable,value,COMBO ID,COMBO Plasma Box Number,COMBO Plasma Box Row (A-H),COMBO Plasma Box Column (1-12),Age,TB Classification,HIV Status,code,tbcode,hivcode,Genes
0,0,A0A075B6I0,C001,12.307254,C001,Peru Plasma Box #1,A,1,5.0,Unlikely TB,Negative,4,4,0,IGLV8-61
1,1,A0A075B6J1-20-123,C001,10.659384,C001,Peru Plasma Box #1,A,1,5.0,Unlikely TB,Negative,4,4,0,IGLV5-37
2,2,A0A075B6J9,C001,11.653405,C001,Peru Plasma Box #1,A,1,5.0,Unlikely TB,Negative,4,4,0,IGLV2-18
3,3,A0A075B6K4,C001,14.075402,C001,Peru Plasma Box #1,A,1,5.0,Unlikely TB,Negative,4,4,0,IGLV3-10
4,4,A0A075B6K4-20-115,C001,11.776722,C001,Peru Plasma Box #1,A,1,5.0,Unlikely TB,Negative,4,4,0,IGLV3-10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
344374,344374,Q9Y5B9,C672,10.539952,C672,PRO South Africa Plasma Box #2,A,11,2.0,Unconfirmed TB,Positive,3,3,1,SUPT16H
344375,344375,Q9Y5C1,C672,12.623133,C672,PRO South Africa Plasma Box #2,A,11,2.0,Unconfirmed TB,Positive,3,3,1,ANGPTL3
344376,344376,Q9Y6R7,C672,12.994101,C672,PRO South Africa Plasma Box #2,A,11,2.0,Unconfirmed TB,Positive,3,3,1,FCGBP
344377,344377,Q9Y6R7-24-5405,C672,11.190325,C672,PRO South Africa Plasma Box #2,A,11,2.0,Unconfirmed TB,Positive,3,3,1,FCGBP
